# Configure model access

In [ ]:
%%capture
!pip install openai
!pip install pandas

In [ ]:
import openai
import os
import json

TOGETHER_API_KEY = 'f9f2c7cc5636ab26066f622d131a151abea0a93a522d3c01fa4f8ee2bcc5e432'
client = openai.OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url="https://api.together.xyz/v1",
)

In [ ]:
def get_code_completion(messages, max_tokens=512, model="meta-llama/Llama-3.3-70B-Instruct-Turbo"):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        stop=[
            "<step>"
        ],
        frequency_penalty=1,
        presence_penalty=1,
        top_p=0.7,
        n=10,
        temperature=0.7,
    )
    return chat_completion


# Basic code completion capabilities

In [ ]:
messages = [
    {
        'role': 'system',
        'content': 'You are an expert programer that helps to write Python code based on the user request, with concise explaination. Do not be too verbose.'
    },
    {
        'role': 'user',
        'content': 'Write a python function to generate the nth fibonacci number.'
    }
]

chat_completion = get_code_completion(messages)
for choice in chat_completion.choices:
    print(choice.message.content)

```python
def fibonacci(n):
    """Generate the nth Fibonacci number."""
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        fib_sequence = [0, 1]
        while len(fib_sequence) < n:
            fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
        return fib_sequence[-1]

# Example usage
print(fibonacci(10))  # Output: 34
```
```python
def fibonacci(n):
    """Return the nth Fibonacci number."""
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        fib_sequence = [0, 1]
        while len(fib_sequence) < n:
            fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
        return fib_sequence[-1]

# Example usage
print(fibonacci(10))  # Output: 34
```
```python
def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer"
    elif n == 1:
    

# Debugging

In [ ]:
messages = [
    {
        'role': 'system',
        'content': 'You are an expert programer that help to review Python code for bugs.'
    },
    {
        'role': 'user',
        'content': """Where is the bug in this code?

        def fibonacci(n):
            if n == 0:
                return 0
            elif n == 1:
                return 1
            else:
                return fibonacci(n-1) + fibonacci(n-2)
        """
    }
]

chat_completion = get_code_completion(messages)
print(chat_completion.choices[0].message.content)

# Step-by-step analysis of the problem:
1. The given code is a recursive implementation of the Fibonacci sequence, which calculates the nth Fibonacci number.
2. **The function does not handle negative inputs**: If `n` is less than 0, the function will enter an infinite recursion and eventually exceed Python's maximum recursion depth, causing a RecursionError.
3. **The function does not handle non-integer inputs**: If `n` is not an integer (e.g., float or string), the function may also enter an infinite recursion or produce incorrect results.

# Fixed solution:
```python
def fibonacci(n):
    """
    Calculate the nth Fibonacci number.

    Args:
        n (int): The index of the Fibonacci number to calculate.

    Returns:
        int: The nth Fibonacci number.

    Raises:
        ValueError: If n is less than 0.
        TypeError: If n is not an integer.
    """
    if not isinstance(n, int):
        raise TypeError("Input must be an integer.")
    if n < 0:
        raise ValueError(

In [ ]:
prompt = """Where is the bug in this code?

        def fibonacci(n):
            if n == 0:
                return 0
            elif n == 1:
                return 1
            else:
                return fibonacci(n-1) + fibonacci(n-2)
        """

# Text-to-SQL generation

In [ ]:
prompt = """
Table department, columns: [dept_no, dept_name]
Table employee, columns: [emp_no, emp_name, dept_no]
Create a MySQL query for all employees in Computer Science Department
""""""

"""

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that creates SQL queries."
    },
    {
        "role": "user",
        "content": prompt
    }
]

chat_completion = get_code_completion(messages)
print(chat_completion.choices[0].message.content)

**MySQL Query: Employees in Computer Science Department**

To retrieve all employees in the Computer Science department, you can use the following SQL query:

```sql
SELECT e.emp_no, e.emp_name
FROM employee e
JOIN department d ON e.dept_no = d.dept_no
WHERE d.dept_name = 'Computer Science';
```

This query works as follows:

1. **Join**: It joins the `employee` table (`e`) with the `department` table (`d`) based on the matching `dept_no` column.
2. **Filter**: It filters the results to only include rows where the `dept_name` is 'Computer Science'.
3. **Select**: It selects only the `emp_no` and `emp_name` columns from the joined tables.

**Example Use Case:**

Suppose you have the following data:

- Table `department`: 
  | dept_no | dept_name        |
  |---------|------------------|
  | D001    | Sales            |
  | D002    | Engineering      |
  | D003    | Computer Science |

- Table `employee`: 
  | emp_no | emp_name   | dept_no |
  |--------|------------|---------|
  | E001  

# Function Calling

# Few-shot prompting

In [ ]:
import pandas as pd

# Sample data for 10 students
data = {
    "Name": ["Alice Johnson", "Bob Smith", "Carlos Diaz", "Diana Chen", "Ethan Clark",
             "Fiona O'Reilly", "George Kumar", "Hannah Ali", "Ivan Petrov", "Julia Müller"],
    "Nationality": ["USA", "USA", "Mexico", "China", "USA", "Ireland", "India", "Egypt", "Russia", "Germany"],
    "Overall Grade": ["A", "B", "B+", "A-", "C", "A", "B-", "A-", "C+", "B"],
    "Age": [20, 21, 22, 20, 19, 21, 23, 20, 22, 21],
    "Major": ["Computer Science", "Biology", "Mathematics", "Physics", "Economics",
             "Engineering", "Medicine", "Law", "History", "Art"],
    "GPA": [3.8, 3.2, 3.5, 3.7, 2.9, 3.9, 3.1, 3.6, 2.8, 3.4]
}

# Creating the DataFrame
students_df = pd.DataFrame(data)

In [ ]:
students_df

,Name,Nationality,Overall Grade,Age,Major,GPA
0,Alice Johnson,USA,A,20,Computer Science,3.8
1,Bob Smith,USA,B,21,Biology,3.2
2,Carlos Diaz,Mexico,B+,22,Mathematics,3.5
3,Diana Chen,China,A-,20,Physics,3.7
4,Ethan Clark,USA,C,19,Economics,2.9
5,Fiona O'Reilly,Ireland,A,21,Engineering,3.9
6,George Kumar,India,B-,23,Medicine,3.1
7,Hannah Ali,Egypt,A-,20,Law,3.6
8,Ivan Petrov,Russia,C+,22,History,2.8
9,Julia Müller,Germany,B,21,Art,3.4


In [ ]:
FEW_SHOT_PROMPT_1 = """
You are given a Pandas dataframe named students_df:
    - Columns: ['Name', 'Nationality', 'Overall Grade', 'Age', 'Major', 'GPA']

User's Question: How to find the youngest student?
"""

FEW_SHOT_ANSWER_1 = """
result = students_df[students_df['Age'] == students_df['Age'].min()]
"""

FEW_SHOT_PROMPT_2 = """
You are given a Pandas dataframe named students_df:
    - Columns: ['Name', 'Nationality', 'Overall Grade', 'Age', 'Major', 'GPA']

User's Question: What are the number of unique majors?
"""

FEW_SHOT_ANSWER_2 = """
result = students_df['Major'].nunique()
"""

FEW_SHOT_PROMPT_USER = """
You are given a Pandas dataframe named students_df:
    - Columns: ['Name', 'Nationality', 'Overall Grade', 'Age', 'Major', 'GPA']

User's Question: How to find the students with GPAs between 3.5 and 3.8?
"""

In [ ]:
messages = [
    {
        "role": "system",
        "content": "Write Pandas code to get the answer to the user's question. Store the answer in a variable named 'result'. Don't include imports. Please wrap the code answer using ``` that takes the DataFrame as input."
    },
    {
        "role": "user",
        "content": FEW_SHOT_PROMPT_1
    },
    {
        "role": "assistant",
        "content": FEW_SHOT_ANSWER_1
    },
    {
        "role": "user",
        "content": FEW_SHOT_PROMPT_2
    },
    {
        "role": "assistant",
        "content": FEW_SHOT_ANSWER_2
    },
    {
        "role": "user",
        "content": FEW_SHOT_PROMPT_USER
    }
]

chat_completion = get_code_completion(messages)
print(chat_completion.choices[0].message.content)

```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```


In [ ]:
for choice in chat_completion.choices:
  print(choice.message.content)

```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```
```python
result = students_df[(students_df['GPA'] >= 3.5) & (students_df['GPA'] <= 3.8)]
```


# Triton Inference Server